In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：用于在线预测的本地图像分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_local_image_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_local_image_classification_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库将本地训练的自定义图像分类模型部署到在线预测中。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已经内置在TensorFlow中。训练的模型会预测出图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 目标

在这个笔记本中，您将在笔记本中创建一个自定义模型，然后学习如何将本地训练好的模型部署到Vertex，然后对部署的模型进行预测。您也可以使用`gcloud`命令行工具或在Google Cloud Console上在线创建和部署模型。

执行的步骤包括：

- 在本地创建模型。
- 在本地训练模型。
- 查看模型评估。
- 将模型上传为Vertex的`Model`资源。
- 将`Model`资源部署到一个serving的`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

### 成本

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
定价](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新版本的 *google-cloud-storage* 库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

重新启动内核

一旦您安装了Vertex客户端库和Google* cloud-storage*，您需要重新启动笔记本内核，以便它能够找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果您有这个选项，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 $300 的免费信用用于支付计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 会将以 `!` 为前缀的行视为 shell 命令，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于本笔记本中的操作。以下是Vertex支持的区域。我们建议您选择距离您最近的区域。

- 美洲: `us-central1`
- 欧洲: `europe-west4`
- 亚太: `asia-east1`

您可能不会使用多区域存储桶来训练Vertex。并非所有区域都支持所有Vertex服务。有关每个区域的最新支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加实时教程会话，您可能正在使用共享的测试帐户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并附加到将在此教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云帐户

**如果您正在使用谷歌云笔记本**，您的环境已经经过身份验证。请跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证，通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务帐户**。

在**服务帐户名称**字段中输入一个名称，然后点击**创建**。

在**授予该服务帐户对项目的访问权限**部分，点击角色下拉列表。在过滤框中键入“Vertex”，选择**Vertex管理员**。在过滤框中键入“存储对象管理员”，选择**存储对象管理员**。

点击创建。一个包含您密钥的 JSON 文件将下载到您的本地环境。

在下面的单元格中将您的服务帐户密钥路径输入为 GOOGLE_APPLICATION_CREDENTIALS 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

当您使用 Vertex 客户端库提交自定义训练作业时，您需要将包含训练代码的 Python 软件包上传到一个云存储桶中。Vertex 将从该软件包中运行代码。在本教程中，Vertex 还会将作业结果生成的训练模型保存在同一个存储桶中。然后，您可以基于该输出创建一个 `Endpoint` 资源，用于提供在线预测。

请在下面设置您的云存储桶的名称。存储桶名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时：运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查Cloud Storage桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

设置变量

接下来，设置一些在教程中使用的变量。
导入库并定义常量。

导入顶点客户端库

将顶点客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

#### 顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、管道和端点服务终点。
- `PARENT`：顶点位置根路径，用于数据集、模型、作业、管道和端点资源。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

硬件加速器

设置用于预测的硬件加速器（例如，GPU）。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个带有4个Nvidia Telsa K80 GPU的容器映像分配给每个VM，您应该指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

对于GPU，可用的加速器包括：
- aip.AcceleratorType.NVIDIA_TESLA_K80
- aip.AcceleratorType.NVIDIA_TESLA_P100
- aip.AcceleratorType.NVIDIA_TESLA_P4
- aip.AcceleratorType.NVIDIA_TESLA_T4
- aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`来使用一个在CPU上运行的容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

### 容器（Docker）映像

接下来，我们将为预测设置Docker容器映像

- 将变量`TF`设置为容器映像的TensorFlow版本。例如，`2-1` 表示版本2.1，`1-15` 表示版本1.15。下面是一些预构建的可用映像列表：

 - TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
 - TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
 - TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
 - TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
 - XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
 - Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请参阅[预构建的预测容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

#### 机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机计算资源。
 - `机器类型`
     - `n1-standard`：每个虚拟CPU 3.75GB内存。
     - `n1-highmem`：每个虚拟CPU 6.5GB内存
     - `n1-highcpu`：每个虚拟CPU 0.9GB内存
 - `虚拟CPU个数`：\[2, 4, 8, 16, 32, 64, 96\]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

#教程

现在你已经准备好开始本地训练一个定制的CIFAR10模型，然后部署到云端。

## 设置客户端

Vertex 客户端库采用客户端/服务器模型工作。在您的一侧（Python 脚本），您将创建一个客户端，该客户端向 Vertex 服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 用于 `Model` 资源的 Model 服务。
- 用于部署的 Endpoint 服务。
- 用于服务的 Prediction 服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

在本教程中，您可以在本地训练一个CIFAR10模型。

### 将位置设置为存储已训练模型

您设置变量`MODEL_DIR`，用于指定在云存储桶中保存 TensorFlow SavedModel 格式模型的位置。

同时，您还需要创建一个用于存储训练脚本的本地文件夹。

In [ ]:
MODEL_DIR = BUCKET_NAME + "/cifar10"
model_path_to_deploy = MODEL_DIR

! rm -rf custom
! mkdir custom
! mkdir custom/trainer

#### Task.py 内容

在下一个单元格中，您可以编写训练脚本 task.py 的内容。我们不会详细讨论，仅供您查看。总结如下：

- 从命令行 (`--model_dir`) 或环境变量 `AIP_MODEL_DIR` 获取保存模型结果的目录。
- 从 TF Datasets (tfds) 加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分发策略。
- 使用参数 `args.epochs` 和 `args.steps` 训练模型（`fit()`）。
- 将训练好的模型保存到指定的模型目录 (`save(args.model_dir)`)。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

### 训练模型

In [ ]:
! python custom/trainer/task.py --epochs=10 --model-dir=$MODEL_DIR

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶中加载它，然后您可以做一些事情，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的 Cloud Storage 路径传递给它 - 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型的表现如何。

### 加载评估数据

您将从 `tf.keras.datasets` 加载 CIFAR10 测试（留存）数据，使用 `load_data()` 方法。这将返回数据集及其标签的元组。第一个元素是训练数据，第二个元素是测试数据。每个元素也是两个元素的元组：图像数据和相应的标签。

您不需要训练数据，因此我们将其加载为 `(_, _)`。

在运行数据进行评估之前，您需要对其进行预处理：

x_test:
1. 通过将每个像素除以 255 来对像素数据进行标准化（重新缩放）。这将用介于 0 和 1 之间的 32 位浮点数替换每个单字节整数像素。

y_test:
2. 标签当前为标量（稀疏）。如果您回顾 `trainer/task.py` 脚本中的 `compile()` 步骤，您会发现它是为稀疏标签编译的。因此我们无需再做任何操作。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 执行模型评估

现在评估一下自定义任务中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

## 上传模型进行服务

接下来，您将从自定义作业上传您的TF.Keras模型到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个服务函数来将数据转换为模型期望的格式。如果您将编码数据发送到Vertex，则您的服务函数会确保在将数据作为输入传递给您的模型之前，在模型服务器上解码数据。

### 服务函数如何工作

当您向在线预测服务器发送请求时，请求会被HTTP服务器接收。HTTP服务器会从HTTP请求内容主体中提取预测请求。提取的预测请求会被转发到服务函数。对于Google预构建的预测容器，请求内容会作为`tf.string`传递给服务函数。

服务函数包括两部分：

- `预处理函数`：
  - 将输入（`tf.string`）转换为基础模型（动态图）的输入形状和数据类型。
  - 执行与训练基础模型期间相同的数据预处理 -- 例如，归一化、缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，制作JSON对象等。

预处理和后处理函数都会被转换为与模型融合的静态图。来自基础模型的输出会传递给后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在为TF.Keras模型构建服务函数时，需要考虑一个重要问题，即它们作为静态图运行。这意味着，您不能使用需要动态图的TF图操作。如果这样做，您将在服务函数的编译过程中收到错误提示，指出您正在使用不受支持的EagerTensor。

### 图像数据的服务功能

要将图像传递给预测服务，您需要将压缩（例如JPEG）图像字节编码为Base64，这样可以在通过网络传输二进制数据时确保内容不被修改。由于部署的模型期望输入数据为原始（未压缩）字节，您需要确保将Base64编码数据转换回原始字节之后再将其作为输入传递给部署的模型。

为了解决这个问题，请定义一个服务功能 (`serving_fn`) 并将其附加到模型作为预处理步骤。添加`@tf.function`装饰器，以便服务功能与底层模型融合在一起（而不是在CPU上游）。

当您发送预测或解释请求时，请求的内容将被解码为一个TensorFlow字符串（`tf.string`）, 然后传递给服务功能 (`serving_fn`)。 服务功能将处理`tf.string`为原始（未压缩）的numpy字节（`preprocess_fn`），以满足模型的输入要求：
- `io.decode_jpeg`- 解压缩 JPG 图像，返回一个具有三个通道（RGB）的TensorFlow张量。
- `image.convert_image_dtype` - 将整数像素值转换为32位浮点数。
- `image.resize` - 调整图像大小以匹配模型的输入形状。
- `resized / 255.0` - 将像素数据重新缩放（归一化）到0到1之间。

在这一点上，数据可以传递给模型 (`m_call`)。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    rescale = tf.cast(resized / 255.0, tf.float32)
    return rescale


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    model, model_path_to_deploy, signatures={"serving_default": serving_fn}
)

获取serving函数签名

您可以通过重新加载模型到内存，并查询与每一层对应的签名来获取模型输入和输出层的签名。

对于您的目的，您需要serving函数的签名。为什么呢？当我们将数据作为HTTP请求包发送进行预测时，图片数据是base64编码的，而我们的TF.Keras模型需要numpy输入。您的serving函数将会把base64转换为一个numpy数组。

在发出预测请求时，您需要将请求路由到serving函数而不是模型，因此您需要知道serving函数的输入层名称 -- 这将在您发出预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

### 上传模型

使用这个辅助函数`upload_model`来上传您的模型，以SavedModel格式存储，上传到`Model`服务，该服务将为您的模型实例化一个Vertex `Model`资源实例。一旦您完成了这一步，您就可以像任何其他Vertex `Model`资源实例一样使用`Model`资源实例，例如部署到用于提供预测的`Endpoint`资源中。

辅助函数接受以下参数：

- `display_name`: 对`Endpoint`服务的一个易读的名称。
- `image_uri`: 模型部署的容器镜像。
- `model_uri`: 指向我们的SavedModel artificat的Cloud Storage路径。对于本教程，这是`trainer/task.py`保存模型文件的Cloud Storage位置，我们在变量`MODEL_DIR`中指定。

辅助函数调用`Model`客户端服务的`upload_model`方法，该方法接受以下参数：

- `parent`: `Dataset`、`Model`和`Endpoint`资源的Vertex根路径。
- `model`: Vertex `Model`资源实例的规范。

现在让我们深入了解Vertex模型规范`model`。这是一个包含以下字段的字典对象：

- `display_name`: `Model`资源的一个易读的名称。
- `metadata_schema_uri`: 由于您的模型是没有使用Vertex `Dataset`资源构建的，您可以将此字段留空(`''`)。
- `artificat_uri`: 存储SavedModel格式的模型的Cloud Storage路径。
- `container_spec`: 这是将安装在`Endpoint`资源上的Docker容器的规范，从而`Model`资源将提供预测。 使用您之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则仅分配CPU。

将模型上传到Vertex Model资源会返回一个长时间运行的操作，因为可能需要一些时间。 您可以调用response.result()，这是一个同步调用，将在Vertex模型资源准备就绪时返回。

辅助函数将返回对应的Vertex Model实例upload_model_response.model的Vertex完全限定标识符。 您将把此标识符保存在变量model_to_deploy_id中，用于后续步骤。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "cifar10-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

### 获取`Model`资源信息

现在让我们只获取您的模型的模型信息。使用这个辅助函数`get_model`，具有以下参数：

- `name`：用于`Model`资源的Vertex唯一标识符。

这个辅助函数调用Vertex`Model`客户端服务的方法`get_model`，具有以下参数：

- `name`：用于`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署“Model”资源

现在部署经过训练的顶点定制“Model”资源。需要两个步骤：

1. 创建一个用于部署“Model”资源的“Endpoint”资源。

2. 将“Model”资源部署到“Endpoint”资源。

### 创建一个 `Endpoint` 资源

使用这个辅助函数 `create_endpoint` 来创建一个端点（Endpoint），用于部署模型以便提供预测服务，带有以下参数：

- `display_name`：`Endpoint` 资源的人类可读名称。

这个辅助函数使用端点客户服务的 `create_endpoint` 方法，该方法接受以下参数：

- `display_name`：`Endpoint` 资源的人类可读名称。

创建一个 `Endpoint` 资源会返回一个长时间运行的操作，因为可能需要一些时间来为提供服务而配置 `Endpoint` 资源。您可以调用 `response.result()`，这是一个同步调用，并且在 Endpoint 资源就绪时会返回。这个辅助函数返回 `Endpoint` 资源的 Vertex 完全限定标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "cifar10_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“终端”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例的扩展

在处理在线预测请求时，您可以选择几种不同的计算实例扩展方式：

- 单个实例：在线预测请求在单个计算实例上处理。
  - 将计算实例的最小 (`MIN_NODES`) 和最大 (`MAX_NODES`) 数量设置为一。

- 手动扩展：在线预测请求在您手动指定的固定数量的计算实例上分配。
  - 将计算实例的最小 (`MIN_NODES`) 和最大 (`MAX_NODES`) 数量设置为相同数量的节点。当模型首次部署到实例时，将配置固定数量的计算实例，并在它们之间均匀分配在线预测请求。

- 自动扩展：在线预测请求在可扩展数量的计算实例上分配。
  - 将计算实例的最小 (`MIN_NODES`) 数量设置为在模型首次部署时需要配置和取消配置的计算实例数量，并将最大 (`MAX_NODES`) 数量设置为根据负载情况需要配置的计算实例数量。

计算实例的最小数量对应于字段`min_replica_count`，最大数量对应于字段`max_replica_count`，在您后续的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用这个辅助函数 `deploy_model` 来部署你在为提供预测创建的 `Model` 资源到 `Endpoint` 资源，使用以下参数：

- `model`：要从训练管道上传（部署）的 Vertex 完全限定的模型标识符。
- `deploy_model_display_name`：部署模型的人类可读名称。
- `endpoint`：要部署模型的 Vertex 完全限定的端点标识符。

辅助函数调用 `Endpoint` 客户端服务的 `deploy_model` 方法，该方法包含以下参数：

- `endpoint`：要部署 `Model` 资源的 Vertex 完全限定的 `Endpoint` 资源标识符。
- `deployed_model`：用于部署模型的要求规范。
- `traffic_split`：在端点上流量进入此模型的百分比，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，请指定为 **{ "0": 100 }**，其中 "0" 指代上传的这个模型，并且 100 表示 100% 的流量。
   - 如果端点上存在要分配流量的现有模型，则使用 `model_id` 指定为 **{ "0": 百分比, model_id: 百分比, ... }**，其中 `model_id` 是已部署端点的模型id。各百分比必须总和为100。

现在让我们深入了解 `deployed_model` 参数。该参数被指定为一个至少包含以下字段的 Python 字典：

- `model`：要部署的 Vertex 完全限定模型标识符。
- `display_name`：部署模型的人类可读名称。
- `disable_container_logging`：禁用容器事件日志记录，例如执行失败（默认情况下启用容器日志记录）。在调试部署时通常启用容器日志记录，然后在生产环境中部署后禁用。
- `dedicated_resources`：用于为服务预测请求扩展的计算实例（副本）数。
  - `machine_spec`：要配置的计算实例。使用之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只会分配 CPU。
  - `min_replica_count`：最初要配置的计算实例数，您之前设置为变量 `MIN_NODES`。
  - `max_replica_count`：要扩展到的最大计算实例数，您之前设置为变量 `MAX_NODES`。

#### 流量分配

现在让我们深入了解 `traffic_split` 参数。该参数被指定为一个 Python 字典。刚开始可能有点令人困惑。让我解释一下，您可以将您的模型的多个实例部署到一个端点，然后设置每个实例获得的流量百分比。

为什么要这么做？也许您已经有一个已部署在生产环境中的先前版本，我们称之为 v1。您在 v2 上得到了更好的模型评估，但在将其部署到生产环境之前，您并不确定它是否确实更好。因此，在流量分配的情况下，您可能希望将 v2 部署到与 v1 同一个端点，但它只获得，例如，10% 的流量。这样，您可以在不干扰大多数用户的情况下监视其表现，直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将通过调用 `response.result()` 来等待操作完成，这将阻塞直到模型被部署。如果这是第一次将模型部署到端点，则可能需要额外几分钟完成资源的配置。

In [ ]:
DEPLOYED_NAME = "cifar10_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

## 发起一个在线预测请求

现在对您部署的模型进行在线预测。

获取测试项目

您将使用数据集的测试（留存）部分中的一个示例作为测试项目。

In [ ]:
test_image = x_test[0]
test_label = y_test[0]
print(test_image.shape)

###准备请求内容
您将以压缩的JPG图像形式发送CIFAR10图像，而不是原始未压缩的字节：

- `cv2.imwrite`：使用OpenCV将未压缩的图像写入磁盘，以压缩的JPEG图像形式。
 - 将图像数据从\[0,1)范围反归一化为\[0,255)范围。
 - 将32位浮点数值转换为8位无符号整数。
- `tf.io.read_file`：将压缩的JPG图像读入内存中，以原始字节形式。
- `base64.b64encode`：将原始字节编码为Base64编码的字符串。

In [ ]:
import base64

import cv2

cv2.imwrite("tmp.jpg", (test_image * 255).astype(np.uint8))

bytes = tf.io.read_file("tmp.jpg")
b64str = base64.b64encode(bytes.numpy()).decode("utf-8")

### 发送预测请求

好的，现在你有一张测试图片。使用这个辅助函数 `predict_image`，它接受以下参数：

- `image`: 作为 numpy 数组的测试图片数据。
- `endpoint`: Vertex 中用于部署 `Model` 资源的 `Endpoint` 资源的完全限定标识符。
- `parameters_dict`: 用于服务的额外参数。

该函数调用预测客户端服务的 `predict` 方法，并使用以下参数：

- `endpoint`: Vertex 中用于部署 `Model` 资源的 `Endpoint` 资源的完全限定标识符。
- `instances`: 要预测的实例（编码图像）的列表。
- `parameters`: 用于服务的额外参数。

为了将图像数据传递给预测服务，在上一步中，你将字节编码为 base64 -- 这样在通过网络传输二进制数据时，内容就不会被篡改。你需要告诉服务二进制在哪里部署了你的模型，以及内容已经经过了 base64 编码，这样它就会在服务二进制端解码。

预测请求中的每个实例都是一个字典条目，格式如下：

                        {serving_input: {'b64': content}}
                      
- `input_name`: 底层模型的输入层的名称。
- `'b64'`: 一个指示内容已经进行了 base64 编码的键。
- `content`: 作为 base64 编码字符串的压缩 JPG 图像字节。

由于 `predict()` 服务可以接受多个图像（实例），你将把你的单个图像发送为一个图像列表。最后一步，你将实例列表打包成 Google 的协议缓冲格式 -- 这就是我们传递给 `predict()` 服务的内容。

`response` 对象返回一个列表，其中列表中的每个元素对应请求中的相应图像。在每个预测的输出中，你将会看到：

- `predictions`: 对每个类别的预测置信度，介于 0 和 1 之间。

In [ ]:
def predict_image(image, endpoint, parameters_dict):
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: {"b64": image}}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters_dict
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_image(b64str, endpoint_id, None)

取消部署`Model`资源

现在取消将您的`Model`资源从提供的`Endpoint`资源中。使用这个辅助函数`undeploy_model`，它接受以下参数：

- `deployed_model_id`：当`Model`资源部署到时，端点服务返回的模型部署标识符。
- `endpoint`：`Model`部署到的`Endpoint`资源的 Vertex 完全限定标识符。

此函数调用端点客户端服务的方法`undeploy_model`，使用以下参数：

- `deployed_model_id`：当`Model`资源被部署时，端点服务返回的模型部署标识符。
- `endpoint`：`Model`资源部署到的`Endpoint`资源的 Vertex 完全限定标识符。
- `traffic_split`：如何在`Endpoint`资源上分配流量给其余的部署模型。

由于这是`Endpoint`资源上唯一部署的模型，您可以通过将其设置为空来留空`traffic_split`。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理工作

要清理此项目中使用的所有GCP资源，您可以[删除您在教程中使用的GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 客制作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME